In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import re

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)


In [3]:
# First look in to the data size
useducation_df.shape

(1492, 25)

In [4]:
# display(useducation_df)

In [5]:
useducation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

**Tasks**
1. Determine all the variable types and find the fraction of the missing values for each variable.
2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?
3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using *only* the observations for that specific year.
4. This time, fill in the missing values using interpolation (extrapolation).
5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

In [6]:
# zero revenue in "STATE_REVENUE" is missing data
useducation_df=useducation_df.replace(0, np.nan)
# Replace 'VI', 'U.S._VIRGIN_ISLANDS', as 'VIRGIN_ISLAND'
useducation_df=useducation_df.replace({'VI':'VIRGIN_ISLANDS', 'U.S._VIRGIN_ISLANDS':'VIRGIN_ISLANDS', \
                                       'AS': 'AMERICAN_SAMOA', 'BI':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BUREAU_OF_INDIAN_EDUCATIO':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BUREAU_OF_INDIAN_EDUCATION': 'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BIE':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'DD': 'DEPARTMENT_OF_DEFENSE', \
                                       'DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY':'DEPARTMENT_OF_DEFENSE', \
                                       '^DOD.*$':'DEPARTMENT_OF_DEFENSE', \
                                       'GU':'GUAM', 'NORTHERN_MARIANAS':'NORTHERN_MARIANA_ISLANDS', \
                                       'PR':'PUERTO_RICO', 'COMMONWEALTH_OF_MARIANAS':'MARIANAS'})

useducation_df.loc[useducation_df['STATE'].str.contains('DOD'),'STATE']='DEPARTMENT_OF_DEFENSE'


In [7]:
useducation_df.isnull().sum()*100/useducation_df.isnull().count()

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          17.627346
TOTAL_REVENUE                   14.209115
FEDERAL_REVENUE                 14.209115
STATE_REVENUE                   16.085791
LOCAL_REVENUE                   14.209115
TOTAL_EXPENDITURE               14.209115
INSTRUCTION_EXPENDITURE         14.209115
SUPPORT_SERVICES_EXPENDITURE    14.209115
OTHER_EXPENDITURE               17.627346
CAPITAL_OUTLAY_EXPENDITURE      14.209115
GRADES_PK_G                     11.729223
GRADES_KG_G                      8.847185
GRADES_4_G                       8.780161
GRADES_8_G                       8.780161
GRADES_12_G                      8.780161
GRADES_1_8_G                     8.780161
GRADES_9_12_G                    8.780161
GRADES_ALL_G                    11.595174
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

##### Primary Key and State are categorical (nominal) variables.  Year is interval variable (continuous) while all others are ratio variable (continuous).  

##### To treat all observations as the same year, we can use mean from the same state to replace the missing value.  For those 'STATE_REVENUE = 0' should be replaced by nan (tossed out).  

In [8]:
useducation_df.nunique()

PRIMARY_KEY                     1487
STATE                             60
YEAR                              26
ENROLL                          1224
TOTAL_REVENUE                   1274
FEDERAL_REVENUE                 1275
STATE_REVENUE                   1250
LOCAL_REVENUE                   1275
TOTAL_EXPENDITURE               1275
INSTRUCTION_EXPENDITURE         1275
SUPPORT_SERVICES_EXPENDITURE    1275
OTHER_EXPENDITURE               1222
CAPITAL_OUTLAY_EXPENDITURE      1275
GRADES_PK_G                     1260
GRADES_KG_G                     1348
GRADES_4_G                      1340
GRADES_8_G                      1347
GRADES_12_G                     1342
GRADES_1_8_G                    1360
GRADES_9_12_G                   1358
GRADES_ALL_G                    1318
AVG_MATH_4_SCORE                 535
AVG_MATH_8_SCORE                 531
AVG_READING_4_SCORE              532
AVG_READING_8_SCORE              497
dtype: int64

The unique value in PRIMARY_KEY is 1487 while there are 1492 rows in PRIMARY_KEY,  indicating 5 replcates.

In [9]:
# show replications
display(useducation_df[useducation_df.PRIMARY_KEY.duplicated(keep='first')])

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
841,2008_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,2008,58191.0,1224312.0,85568.0,NaN,1138744.0,1224785.0,520090.0,...,3292.0,2936.0,2092.0,26249.0,10315.0,36676.0,NaN,NaN,NaN,NaN
880,2008_VIRGINIA,VIRGINIA,2008,1230857.0,14768120.0,897627.0,5957786.0,7912707.0,15236306.0,8003167.0,...,4666.0,3863.0,2535.0,36807.0,12837.0,51345.0,NaN,NaN,NaN,NaN
882,2008_VIRGINIA,VIRGINIA,2008,1230857.0,14768120.0,897627.0,5957786.0,7912707.0,15236306.0,8003167.0,...,2548.0,1485.0,484.0,19226.0,2758.0,24554.0,NaN,NaN,NaN,NaN
896,2009_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,2009,44331.0,801008.0,75856.0,NaN,725152.0,805362.0,330698.0,...,3423.0,2860.0,2188.0,26762.0,9970.0,36823.0,219.260141,253.595187,201.984638,252.631381
949,2010_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,2010,43866.0,1195934.0,80585.0,NaN,1115349.0,1290048.0,526469.0,...,3397.0,2992.0,2203.0,27264.0,9977.0,NaN,NaN,NaN,NaN,NaN


In [10]:
# drop replications
useducation_df.drop_duplicates(subset='PRIMARY_KEY', inplace= True)

In [11]:
# # Group by state
# useducation2_df = useducation_df.drop(columns = 'YEAR').groupby('STATE').mean()

# Replace by state mean

column_list = ['ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE', \
               'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE', \
               'CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G', 'GRADES_12_G', \
               'GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', \
               'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']


# useducation2_df[column_list]= useducation2_df.groupby("STATE").transform(lambda x: x.fillna(x.mean()))[column_list]

useducation2_df=useducation_df.copy()

for column in column_list:
    useducation2_df.loc[:, column].fillna(useducation2_df[column].mean(), inplace=True)

In [12]:
## display(useducation2_df)

##### Take year into consideration

In [13]:
useducation3_df=useducation_df.copy()
years = useducation3_df.YEAR.unique()

for column in column_list:
    for year in years:
        useducation3_df.loc[useducation3_df.YEAR==year, column].fillna( \
            useducation3_df[useducation3_df.YEAR==year][column].mean(), inplace=True)

##### Interpolate

In [14]:
useducation4_df=useducation_df.copy()
for column in column_list:
    useducation4_df.loc[:,column].interpolate(inplace=True)


In [15]:
##### Compare different methods

In [16]:
for column in column_list:
    df = pd.concat([useducation_df[column], useducation2_df[column], useducation3_df[column], useducation4_df[column]], axis =1)
    print(df.describe())

             ENROLL        ENROLL        ENROLL        ENROLL
count  1.224000e+03  1.487000e+03  1.224000e+03  1.451000e+03
mean   9.175416e+05  9.175416e+05  9.175416e+05  1.007653e+06
std    1.066514e+06  9.675434e+05  1.066514e+06  1.009922e+06
min    4.386600e+04  4.386600e+04  4.386600e+04  4.386600e+04
25%    2.645145e+05  3.169350e+05  2.645145e+05  3.008390e+05
50%    6.499335e+05  8.222450e+05  6.499335e+05  7.563190e+05
75%    1.010532e+06  9.209985e+05  1.010532e+06  1.595024e+06
max    6.307022e+06  6.307022e+06  6.307022e+06  6.307022e+06
       TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE
count   1.275000e+03   1.487000e+03   1.275000e+03   1.487000e+03
mean    9.102045e+06   9.102045e+06   9.102045e+06   1.112698e+07
std     1.175962e+07   1.088851e+07   1.175962e+07   1.201250e+07
min     4.656500e+05   4.656500e+05   4.656500e+05   4.656500e+05
25%     2.189504e+06   2.566118e+06   2.189504e+06   2.566118e+06
50%     5.085826e+06   6.364028e+06   5.085826

         GRADES_12_G    GRADES_12_G    GRADES_12_G    GRADES_12_G
count    1356.000000    1487.000000    1356.000000    1487.000000
mean    54462.023599   54462.023599   54462.023599   50424.839274
std     67939.693855   64875.964643   67939.693855   66355.439930
min       311.000000     311.000000     311.000000     311.000000
25%     11047.250000   13869.000000   11047.250000    9099.000000
50%     36830.500000   39799.000000   36830.500000   33478.000000
75%     67488.250000   64689.000000   67488.250000   65311.500000
max    498403.000000  498403.000000  498403.000000  498403.000000
       GRADES_1_8_G  GRADES_1_8_G  GRADES_1_8_G  GRADES_1_8_G
count  1.356000e+03  1.487000e+03  1.356000e+03  1.487000e+03
mean   5.210280e+05  5.210280e+05  5.210280e+05  4.829727e+05
std    6.441294e+05  6.150825e+05  6.441294e+05  6.290764e+05
min    4.878000e+03  4.878000e+03  4.878000e+03  4.878000e+03
25%    1.116368e+05  1.241655e+05  1.116368e+05  8.294550e+04
50%    3.480880e+05  4.027180e+05 

##### Conclusion: Method2 gives overestimate because the missing values are mostly small states, which is replaced by large states.